In [8]:
import spacy

In [9]:
import torchtext

In [10]:
# nlp = spacy.load('en_core_web_sm')

In [11]:
spacy.load('en')

In [12]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

In [13]:
torch.cuda.is_available()

True

In [14]:
# torch.cuda.device_count()
# torch.cuda.get_device_capability(0)
# torch.cuda.get_device_name(0)
# torch.backends.cudnn.enabled
# torch.backends.cudnn.version()
torch.cuda.device(0)

In [15]:
import torch; print(torch.rand(2,3).cuda())


 0.3074  0.4159  0.1509
 0.7400  0.8667  0.8565
[torch.cuda.FloatTensor of size 2x3 (GPU 0)]



In [16]:
torch.cuda.set_device(0)

## Language modeling

### Data

In [10]:
train = pd.read_csv("C://Users/hafid/Dropbox/3.SelfStudy/kaggle/ai6-kaggle-toxic/data/train/train.csv")
test = pd.read_csv("C://Users/hafid/Dropbox/3.SelfStudy/kaggle/ai6-kaggle-toxic/data/test/test.csv")
# subm = pd.read_csv("C://Users/hafid\Dropbox/3.SelfStudy/kaggle/jigsaw-toxic-comment-classification/sample_submission.csv")

In [74]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
train.comment_text[0]

In [12]:
PATH='C://Users/hafid/Dropbox/3.SelfStudy/kaggle/ai6-kaggle-toxic/'

TRN_PATH = 'data/train/'
VAL_PATH = 'data/test/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

os.listdir(PATH)

['.git',
 '.ipynb_checkpoints',
 'data',
 'Getting started with Pytorch.ipynb',
 'models',
 'README.md',
 'src',
 'submission',
 'tmp']

We use Pytorch's [torchtext](https://github.com/pytorch/text) library to preprocess our data, telling it to use the wonderful [spacy](https://spacy.io/) library to handle tokenization.

First, we create a torchtext *field*, which describes how to preprocess a piece of text - in this case, we tell torchtext to make everything lowercase, and tokenize it with spacy.

In [22]:
TEXT = data.Field(lower=True, tokenize=spacy_tok)

fastai works closely with torchtext. We create a ModelData object for language modeling by taking advantage of `LanguageModelData`, passing it our torchtext field object, and the paths to our training, test, and validation sets. In this case, we don't have a separate test set, so we'll just use `VAL_PATH` for that too.

As well as the usual `bs` (batch size) parameter, we also not have `bptt`; this define how many words are processing at a time in each row of the mini-batch. More importantly, it defines how many 'layers' we will backprop through. Making this number higher will increase time and memory requirements, but will improve the model's ability to handle long sentences.

What does spacy_tok output look like?


In [23]:
' '.join(spacy_tok(train.comment_text[0]))

"Explanation . Why the edits made under my username Hardcore Metallica Fan were reverted ? They were n't vandalisms , just closure on some GAs after I voted at New York Dolls FAC . And please do n't remove the template from the talk page since I 'm retired now.89.205.38.27"

It seems that spacy_tok does not remove linefeed. We'll need to take care of that.

In [17]:
train.comment_text = train.comment_text.replace('\n','. ', regex=True)
test.comment_text = test.comment_text.replace('\n','. ', regex=True)

In [18]:
train.comment_text[0]

"Explanation. Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [19]:
test.comment_text[0]

"Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,"

In [20]:
bs=64; bptt=80

In [42]:
# FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
FILES = dict(train_df=train, val_df=test, test_df=test)
md = LanguageModelData.from_dataframes(PATH,TEXT, col="comment_text", **FILES, bs=bs, bptt=bptt, min_freq=10)
# md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

After building our `ModelData` object, it automatically fills the `TEXT` object with a very important attribute: `TEXT.vocab`. This is a *vocabulary*, which stores which words (or *tokens*) have been seen in the text, and how each word will be mapped to a unique integer id. We'll need to use this information again later, so we save it.

*(Technical note: python's standard `Pickle` library can't handle this correctly, so at the top of this notebook we used the `dill` library instead and imported it as `pickle`)*.

In [43]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

Here are the: # batches; # unique tokens in the vocab; # tokens in the training set; # sentences

In [44]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(2660, 27242, 1, 13628745)

This is the start of the mapping from integer IDs to unique tokens.

In [45]:
# 'itos': 'int-to-string'
TEXT.vocab.itos[:12]

['<unk>', '<pad>', '.', 'the', ',', '"', 'to', 'i', 'of', 'and', 'you', 'a']

In [46]:
# 'stoi': 'string to int'
TEXT.vocab.stoi['the']

3

Note that in a `LanguageModelData` object there is only one item in each dataset: all the words of the text joined together.

In [47]:
md.trn_ds[0].text[:12]

['explanation',
 '.',
 'why',
 'the',
 'edits',
 'made',
 'under',
 'my',
 'username',
 'hardcore',
 'metallica',
 'fan']

torchtext will handle turning this words into integer IDs for us automatically.

In [48]:
TEXT.numericalize([md.trn_ds[0].text[:12]])

Variable containing:
   686
     2
    92
     3
   146
   149
   195
    43
   672
  4400
 11375
  1254
[torch.cuda.LongTensor of size 12x1 (GPU 0)]

Our `LanguageModelData` object will create batches with 64 columns (that's our batch size), and varying sequence lengths of around 80 tokens (that's our `bptt` parameter - *backprop through time*).

Each batch also contains the exact same data as labels, but one word later in the text - since we're trying to always predict the next word. The labels are flattened into a 1d array.

In [49]:
next(iter(md.trn_dl))

(Variable containing:
    686     44   7702  ...      35     14   1009
      2    378     94  ...     201   1312      2
     92      6  10997  ...      65  10179     13
         ...            ⋱           ...         
   8823   6064      2  ...      92   4777      0
     25     13      9  ...     187   5965      5
    199     25  14647  ...      88      0      5
 [torch.cuda.LongTensor of size 78x64 (GPU 0)], Variable containing:
      2
    378
     94
   ⋮   
     39
      8
    143
 [torch.cuda.LongTensor of size 4992 (GPU 0)])

### Train

We have a number of parameters to set - we'll learn more about these later, but you should find these values suitable for many problems.

In [50]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

Researchers have found that large amounts of *momentum* (which we'll learn about later) don't work well with these kinds of *RNN* models, so we create a version of the *Adam* optimizer with less momentum than it's default of `0.9`.

In [51]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

fastai uses a variant of the state of the art [AWD LSTM Language Model](https://arxiv.org/abs/1708.02182) developed by Stephen Merity. A key feature of this model is that it provides excellent regularization through [Dropout](https://en.wikipedia.org/wiki/Convolutional_neural_network#Dropout). There is no simple way known (yet!) to find the best values of the dropout parameters below - you just have to experiment...

However, the other parameters (`alpha`, `beta`, and `clip`) shouldn't generally need tuning.

In [52]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

As you can see below, I gradually tuned the language model in a few stages. I possibly could have trained it further (it wasn't yet overfitting), but I didn't have time to experiment more. Maybe you can see if you can train it to a better accuracy! (I used `lr_find` to find a good learning rate, but didn't save the output in this notebook. Feel free to try running it yourself now.)

In [57]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      4.595523   4.398488  
    1      4.308832   4.16604                                                                                          
    2      4.180147   4.073452                                                                                         
    3      4.188107   4.092638                                                                                         
    4      4.090945   4.016657                                                                                         
    5      4.015669   3.959933                                                                                         
    6      3.98171    3.942955                                                                                         
    7      4.106762   4.024378                                                                                         
    8  

[3.874336]

In [58]:
learner.save_encoder('adam1_enc')

In [93]:
# learner.load_encoder('adam1_enc')

In [59]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=10)

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

epoch      trn_loss   val_loss                                                                                         
    0      4.02468    3.979456  
    1      4.010671   3.952216                                                                                         
    2      3.992484   3.936517                                                                                         
    3      3.968075   3.916682                                                                                         
    4      3.93434    3.896811                                                                                         
    5      3.890588   3.880113                                                                                         
    6      3.850605   3.862473                                                                                         
    7      3.834424   3.850614                                                                                         
    8  

[3.8437555]

In [60]:
learner.save_encoder('adam3_10_enc')

In the sentiment analysis section, we'll just need half of the language model - the *encoder*, so we save that part.

Language modeling accuracy is generally measured using the metric *perplexity*, which is simply `exp()` of the loss function we used.

In [ ]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [61]:
math.exp(3.8437555)

46.70052936268467

### Test

We can play around with our language model a bit to check it seems to be working OK. First, let's create a short bit of text to 'prime' a set of predictions. We'll use our torchtext field to numericalize it so we can feed it to our language model.

In [62]:
m=learner.model
ss=""". So, it wasn't quite was I was expecting, but I really liked it anyway! The best"""
s = [spacy_tok(ss)]
t=TEXT.numericalize(s)
' '.join(s[0])

". So , it was n't quite was I was expecting , but I really liked it anyway ! The best"

We haven't yet added methods to make it easy to test a language model, so we'll need to manually go through the steps.

In [63]:
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=bs

Let's see what the top 10 predictions were for the next word after our short text:

In [ ]:
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

...and let's see if our model can generate a bit more text all by itself!

In [65]:
print(ss,"\n")
for i in range(50):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))
print('...')

. So, it wasn't quite was I was expecting, but I really liked it anyway! The best 

wishes to be a good editor .   . .   the article is not a place for advertising or a collection of external links .   please do not add nonsense to wikipedia . it is considered vandalism . if you would like to experiment , use the sandbox ...
